# PRV Functions

In [1]:
import numpy as np

# 1) PRV_to_DCM

In [2]:
def PRV_to_DCM(e, phi_deg):
    """
    Converts a Principal Rotation Vector (PRV) to a rotation matrix.

    Args:
        e (np.array)   : The unit vector of the PRV.
        phi_deg (float): The rotation angle of the PRV in degrees.

    Returns:
        np.array: A 3x3 rotation matrix.
    """
    # Convert the angle from degrees to radians
    phi_rad = np.radians(phi_deg)
    
    # Calculate the cosine and sine of the angle
    c_phi = np.cos(phi_rad)
    s_phi = np.sin(phi_rad)
    
    # Calculate the matrix Sigma
    Sigma = 1 - c_phi

    # Ensure e is a float array to avoid UFuncTypeError during in-place operations
    e = np.array(e, dtype=float)

    # Normalize e vector to ensure it's a valid unit vector
    e /= np.linalg.norm(e)
    
    # Decompose the unit vector into its components
    e1, e2, e3 = e
    
    # Construct the rotation matrix using the given formula
    C = np.array([[((e1**2)*Sigma + c_phi), (e1*e2*Sigma + e3*s_phi), (e1*e3*Sigma - e2*s_phi)],
                  [(e2*e1*Sigma - e3*s_phi), ((e2**2)*Sigma + c_phi), (e2*e3*Sigma + e1*s_phi)],
                  [(e3*e1*Sigma + e2*s_phi), (e3*e2*Sigma - e1*s_phi), ((e3**2)*Sigma + c_phi)]])

    return C

In [6]:
axis = [3, 2, 3]  
angle = 90
C = PRV_to_DCM(axis, angle)
C

array([[ 0.40909091,  0.91232942, -0.01731052],
       [-0.36687488,  0.18181818,  0.91232942],
       [ 0.83549234, -0.36687488,  0.40909091]])

# 2) DCM_to_PRV

In [ ]:
def DCM_to_PRV(C):
    """
    Converts a rotation matrix to a Principal Rotation Vector (PRV).

    Args:
        C (np.array): A 3x3 rotation matrix.

    Returns:
        tuple: A PRV represented as (e_vector, phi_angle), where e_vector is the
               rotation axis (unit vector), and phi_angle is the rotation angle in degrees.
    """
    # Compute the angle phi from the trace of the rotation matrix
    trace_C = np.trace(C)
    phi = np.arccos((trace_C - 1) / 2)

    # Ensure phi is in the range [0, pi]
    #phi = np.clip(phi, 0, np.pi)

    if np.isclose(phi, 0):
        e = np.diag(C).copy()
    
    # Handle edge cases where phi is close to pi
    elif np.isclose(phi, np.pi):
        # For phi = π, special handling is required to find the rotation axis.
        # Compute (C + I)
        CpI = C + np.identity(3)
        #print(C)
        #print(CpI)

        # Extract the axis from the non-zero rows of (C + I)
        # Due to numerical errors, we need to find the largest column in CpI
        norms = np.linalg.norm(CpI, axis=0)
        #print(norms)

        index = np.argmax(norms)

        e = CpI[:, index]
    
    else:
        # Compute the rotation axis using the standard formula
        e = (1 / (2 * np.sin(phi))) * np.array([C[2, 1] - C[1, 2],
                                                C[0, 2] - C[2, 0],
                                                C[1, 0] - C[0, 1]
        ])
    
    # Normalize the axis vector and convert phi to degrees
    e /= np.linalg.norm(e)
    phi = np.rad2deg(phi)

    return e, phi

In [ ]:
DCM_to_PRV(C)